Les données de temperature sont pris de https://climat.meteo.gc.ca/index_f.html

In [15]:
import json
import pandas as pd

In [ ]:
test = [{"time":i, "value":i*2} for i in range(10)]
# tmp = json.dumps(test) #convert to json

labels = [i for i in range(100)]
temp_values = [i*2 for i in range(100)]
hum_values = [20-i*0.3 for i in range(100)]
water_values = [60-i*0.5 for i in range(100)]
pH_values = [6-0.005*i for i in range(100)]
CO2_values = [800-0.04*i for i in range(100)]
Sun_values = [600+10*i for i in range(100)]

test = {"labels":labels,"temp_values":temp_values,"hum_values":hum_values,"water_values":water_values,"pH_values":pH_values,"CO2_values":CO2_values,"Sun_values":Sun_values}

print(test)
# print(tmp)
with open('data.json', 'w') as fp:
    json.dump(test, fp)

In [60]:
temps_data = pd.read_csv("fr_climat_horaires_QC_702S006_07-2022_P1H.csv",decimal=',')
temps_data = temps_data[["Jour", "Heure (HNL)","Temp (°C)","Hum. rel (%)","Hauteur de précip. (mm)"]]
temps_data["Heure (HNL)"] = (temps_data["Heure (HNL)"].str.replace(":00","")).astype(int)
# temps_data

In [58]:
day_temp_data = temps_data.groupby("Jour").mean(numeric_only=True)
# day_temp_data

In [59]:
times = []
temp_values = [] 
hum_values = []
water_values = []
pH_values = []
co2_values = []
sun_values = []

time_len = [3600,25,7]
for j in time_len:
    times.append([i for i in range(j)])
    temp_values.append([i*2 for i in range(j)])
    hum_values.append([20-i*0.3 for i in range(j)])
    water_values.append([60-i*0.5 for i in range(j)])
    pH_values.append([6-0.005*i for i in range(j)])
    co2_values.append([800-0.04*i for i in range(j)])
    sun_values.append([600+10*i for i in range(j)])

test = {"times":times,"temp_values":temp_values,"hum_values":hum_values,"water_values":water_values,"pH_values":pH_values,"CO2_values":co2_values,"Sun_values":sun_values}
# print(test)
# print(tmp)
with open('data.json', 'w') as fp:
    json.dump(test, fp)